# Assignment 9

### Shovan Biswas

### Data Set Information

Source: http://archive.ics.uci.edu/ml/datasets/Spambase

The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...

Our collection of spam e-mails came from our postmaster and individuals who had filed spam. Our collection of non-spam e-mails came from filed work and personal e-mails, and hence the word 'george' and the area code '650' are indicators of non-spam. These are useful when constructing a personalized spam filter. One would either have to blind such non-spam indicators or get a very wide collection of non-spam to generate a general purpose spam filter.

For background on spam:

Cranor, Lorrie F., LaMacchia, Brian A. Spam!
Communications of the ACM, 41(8):74-83, 1998.

(a) Hewlett-Packard Internal-only Technical Report. External forthcoming.
(b) Determine whether a given email is spam or not.
(c) ~7% misclassification error. False positives (marking good mail as spam) are very undesirable.If we insist on zero false positives in the training/testing set, 20-25% of the spam passed through the filter.

#### Data assembling

From source website, we donwloaded files spambase.names & spambase.data. The latter contains the actual data and the former contains the column names and the directions to construct the file. Following the directions, we created spambase.csv.    

### Import and read    

In [191]:
import nltk
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
import sklearn.metrics as sm

import seaborn as sn

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt

In [192]:
spam_data = pd.read_csv("spambase.csv")

### Basic data exploration   

In [193]:
spam_data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spamclass
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [194]:
print("Content of this dataset: " + str(spam_data.shape[0]) + " rows and " + str(spam_data.shape[1]) + " columns")

Content of this dataset: 4601 rows and 58 columns


Following columns were mentioned in the file spambase.names.    

In [195]:
print(spam_data.dtypes)

word_freq_make                float64
word_freq_address             float64
word_freq_all                 float64
word_freq_3d                  float64
word_freq_our                 float64
word_freq_over                float64
word_freq_remove              float64
word_freq_internet            float64
word_freq_order               float64
word_freq_mail                float64
word_freq_receive             float64
word_freq_will                float64
word_freq_people              float64
word_freq_report              float64
word_freq_addresses           float64
word_freq_free                float64
word_freq_business            float64
word_freq_email               float64
word_freq_you                 float64
word_freq_credit              float64
word_freq_your                float64
word_freq_font                float64
word_freq_000                 float64
word_freq_money               float64
word_freq_hp                  float64
word_freq_hpl                 float64
word_freq_ge

#### Column wise statistics of data    

In [196]:
spam_data.describe()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spamclass
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


#### Spam / non-spam count     

The last column spamclass determines whether a data is spam or otherwise. If spamclass is 1, it's a spam and if it's 0, it's not a spam. (Ref: attribute descriptions in spambase.names).       

In [197]:
spam_column = len(spam_data[spam_data.spamclass == 1])
notspam_column = len(spam_data[spam_data.spamclass == 0])

print("Spam data count: %d" %spam_column)
print("Non spam data count: %d" %notspam_column)

Spam data count: 1813
Non spam data count: 2788


#### Check for nulls 

In [198]:
spam_data[spam_data.isnull().any(axis = 1)]

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spamclass


### Data Preparation

We'll split the data into train and test, distributing 70% in train.         
(Fact: **spam_data** contains the entire data that was read)

In [199]:
train_percentage = 0.7
val_percentage = 0.15

spam_data_rows = len(spam_data)
train_rows = int(spam_data_rows * train_percentage)
val_rows = int(spam_data_rows * val_percentage)
test_rows = spam_data_rows - train_rows - val_rows

In [200]:
print("Training rows (70 prc of total): %d" %train_rows)
print("Validation rows (15 prc of total): %d" %val_rows)
print("Testing rows (15 prc of total): %d" %test_rows)
print("Total: %d" %(train_rows + val_rows + test_rows))

Training rows (70 prc of total): 3220
Validation rows (15 prc of total): 690
Testing rows (15 prc of total): 691
Total: 4601


In [201]:
train_set, test_set = train_test_split(spam_data, test_size = test_rows, random_state = 8)
train_set, val_set = train_test_split(train_set, test_size = val_rows, random_state = 88)

In [202]:
print("Training set: %d" %len(train_set))
print("Validation set: %d" %len(val_set))
print("Testing set: %d" %len(test_set))
print("Total: %d" %(len(train_set) + len(val_set) + len(test_set)))

Training set: 3220
Validation set: 690
Testing set: 691
Total: 4601


### Confusion Matrix

The following function (just a function, no action is happening here) computes Confusion Matrix, which shows True Positives (TP), True Negatives (TN), False Positives (FP) and False Negatives (FN), from which the "Accuracy" of an algorithm can be determined.    

In [203]:
def func_confusion_matrix(y_true, y_pred):
    cm = sm.confusion_matrix(y_true, y_pred, labels = [1, 0])
    print("TP: %d" %cm[0,0])
    print("FP: %d" %cm[1,0])
    print("TN: %d" %cm[1,1])
    print("FN: %d" %cm[0,1])
    print(sm.classification_report(y_true, y_pred, labels = [1,0], target_names = ["Spam", "Not spam"]))  
    return cm

### Data Analysis

We'll classify the data in three possible machine learning methods, namely Decision Tree, Random Forest and Support Vector Machines.      

#### Decision Tree

First, we'll classify using decision tree, where each node represents data, whether it's spam or not.    
(Fact: **spamclass** is the spam/not spam indicator column in table)

In [204]:
tree = tree.DecisionTreeClassifier(criterion = "entropy", random_state = 88)

#### Classification and model statistics for training set

In [205]:
train_class = train_set['spamclass']
train_vars = train_set.drop(labels = 'spamclass', axis = 1)

tree_fit = tree.fit(train_vars, train_class)

tree_train = tree_fit.predict(train_vars)
cm = func_confusion_matrix(train_class, tree_train)

TP: 1260
FP: 0
TN: 1958
FN: 2
              precision    recall  f1-score   support

        Spam       1.00      1.00      1.00      1262
    Not spam       1.00      1.00      1.00      1958

    accuracy                           1.00      3220
   macro avg       1.00      1.00      1.00      3220
weighted avg       1.00      1.00      1.00      3220



#### Classification and model statistics for validation set

In [206]:
validate_class = val_set['spamclass']
validate_vars = val_set.drop(labels='spamclass', axis = 1)

tree_validate = tree_fit.predict(validate_vars)     # tree = tree.DecisionTreeClassifier(..) was done once above.
cm = func_confusion_matrix(validate_class, tree_validate)

TP: 252
FP: 17
TN: 401
FN: 20
              precision    recall  f1-score   support

        Spam       0.94      0.93      0.93       272
    Not spam       0.95      0.96      0.96       418

    accuracy                           0.95       690
   macro avg       0.94      0.94      0.94       690
weighted avg       0.95      0.95      0.95       690



#### Classification and model statistics for test set

In [207]:
test_class = test_set['spamclass']
test_vars = test_set.drop(labels='spamclass', axis = 1)

tree_test = tree_fit.predict(test_vars)             # tree = tree.DecisionTreeClassifier(..) was done once.
cm = func_confusion_matrix(test_class, tree_test)

TP: 251
FP: 29
TN: 383
FN: 28
              precision    recall  f1-score   support

        Spam       0.90      0.90      0.90       279
    Not spam       0.93      0.93      0.93       412

    accuracy                           0.92       691
   macro avg       0.91      0.91      0.91       691
weighted avg       0.92      0.92      0.92       691



#### Determine feature importance

In [210]:
def func_arrange_feature_by_importance(fit, vars):
    df = {'variable': pd.Series(vars.columns.values), 'imp': pd.Series(fit.feature_importances_)}
    return pd.DataFrame(df, columns=['variable','imp']).sort_values(['imp'], ascending=0).head(10)

func_arrange_feature_by_importance(tree_fit, test_vars)

,variable,imp
51,char_freq_!,0.296289
52,char_freq_$,0.130370
6,word_freq_remove,0.125547
54,capital_run_length_average,0.056054
55,capital_run_length_longest,0.035736
26,word_freq_george,0.033155
56,capital_run_length_total,0.026242
24,word_freq_hp,0.025670
4,word_freq_our,0.025391
18,word_freq_you,0.024984


#### Random Forest

Random Forest being a collection of Decision Trees, is more complex. While Decision Tree is implemented on an entire dataset, using all features (variables) of interest, a Random Forest randomly selects observations (i.e. rows) and specific features to build multiple Decision Trees and computes the averages.

In [176]:
forest = ensemble.RandomForestClassifier(criterion = "entropy", random_state = 88)

#### Classification and model statistics for training set

In [177]:
forest_fit = forest.fit(train_vars, train_class)

forest_train = forest_fit.predict(train_vars)
cm = func_confusion_matrix(train_class, forest_train)

TP: 1248
FP: 3
TN: 1955
FN: 14
              precision    recall  f1-score   support

        Spam       1.00      0.99      0.99      1262
    Not spam       0.99      1.00      1.00      1958

    accuracy                           0.99      3220
   macro avg       1.00      0.99      0.99      3220
weighted avg       0.99      0.99      0.99      3220



#### Classification and model statistics for test set

In [178]:
forest_test = forest_fit.predict(test_vars)
cm = func_confusion_matrix(test_class, forest_test)

TP: 248
FP: 16
TN: 396
FN: 31
              precision    recall  f1-score   support

        Spam       0.94      0.89      0.91       279
    Not spam       0.93      0.96      0.94       412

    accuracy                           0.93       691
   macro avg       0.93      0.93      0.93       691
weighted avg       0.93      0.93      0.93       691



In [179]:
func_arrange_feature_by_importance(forest_fit, test_vars)

,variable,imp
51,char_freq_!,0.133634
52,char_freq_$,0.110833
55,capital_run_length_longest,0.078106
15,word_freq_free,0.066308
54,capital_run_length_average,0.065711
6,word_freq_remove,0.053635
24,word_freq_hp,0.041705
56,capital_run_length_total,0.036781
25,word_freq_hpl,0.034489
18,word_freq_you,0.033800


#### Support Vector Machines

Support Vector Machine (SVM) is a machine learning algorithms that is used to separate sets of points in a hyperspace, by a hyperplain.

In [186]:
svm = svm.SVC(random_state = 88)

In [187]:
svm_fit = svm.fit(train_vars, train_class)

svm_train = svm_fit.predict(train_vars)
cm = func_confusion_matrix(train_class, svm_train)

TP: 1126
FP: 52
TN: 1906
FN: 136
              precision    recall  f1-score   support

        Spam       0.96      0.89      0.92      1262
    Not spam       0.93      0.97      0.95      1958

    accuracy                           0.94      3220
   macro avg       0.94      0.93      0.94      3220
weighted avg       0.94      0.94      0.94      3220



In [188]:
svm_test = svm_fit.predict(test_vars)
cm = func_confusion_matrix(test_class, svm_test)

TP: 214
FP: 61
TN: 351
FN: 65
              precision    recall  f1-score   support

        Spam       0.78      0.77      0.77       279
    Not spam       0.84      0.85      0.85       412

    accuracy                           0.82       691
   macro avg       0.81      0.81      0.81       691
weighted avg       0.82      0.82      0.82       691

